# Persistence and Execution at scale via GPO scheduled task
Detect lateral movement using GPO scheduled task, ususally used to deploy ransomware at scale

## Rule Content
```
- title: Persistence and Execution at scale via GPO scheduled task
  id: a8f29a7b-b137-4446-80a0-b804272f3da2
  description: Detect lateral movement using GPO scheduled task, ususally used to
    deploy ransomware at scale
  author: Samir Bousseaden
  references:
  - https://twitter.com/menasec1/status/1106899890377052160
  tags:
  - attack.persistence
  - attack.lateral_movement
  - attack.t1053
  logsource:
    product: windows
    service: security
    description: The advanced audit policy setting "Object Access > Audit Detailed
      File Share" must be configured for Success/Failure
    category: null
  detection:
    selection:
      EventID: 5145
      ShareName: \\*\SYSVOL
      RelativeTargetName: '*ScheduledTasks.xml'
      Accesses: '*WriteData*'
    condition: selection
  falsepositives:
  - if the source IP is not localhost then it's super suspicious, better to monitor
    both local and remote changes to GPO scheduledtasks
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"5145" AND share_name.keyword:\\*\\SYSVOL AND share_relative_target_name.keyword:*ScheduledTasks.xml AND Accesses.keyword:*WriteData*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()